In [1]:
import openai
import json
from IPython.display import JSON
import sys 
#sys.path.append("embedding-for-search")
from readProp import *

In [2]:
openai.api_key =  readProperties('OPENAI_KEY')

In [3]:
color_prefix_by_role = {
    "system": "\033[0m",  # gray
    "user": "\033[0m",  # gray
    "assistant": "\033[34m",  # blue,
    "function": "\033[35m" # magenta
}


def print_messages(messages, color_prefix_by_role=color_prefix_by_role) -> None:
    """Prints messages sent to or from LLM."""
    for message in messages:
        role = message["role"]
        color_prefix = color_prefix_by_role[role]
        content = message["content"]
        print(f"{color_prefix}\n[{role}]\n{content}")

In [4]:
def get_sales_campaign_running(city_name:str,bank_name:str,detailed_reason:str):
    """returns the sales campaign which are currently happening in city
       This is internally managed by sales team . 
       We are simulating call to sales system 
 
       
    """
    campaigns = {
        "Mumbai" : 
        [
            {    
            "from_date" : "01-nov-2023",
            "to_date"   : "03-nov-2023",
            "description" : "cross border payments cheaper and faster",
            "sales_team_status" : "busy"
            } ,
            {
            "from_date" : "22-nov-2023",
            "to_date"   : "23-nov-2023",
            "description" : "blockchain payments at fingertips",
            "sales_team_status" : "busy"
            }
        ],
        "Bangalore" :  
        [
            {    
            "from_date" : "05-nov-2023",
            "to_date"   : "08-nov-2023",
            "description" : "syndicated loans and blockchain",
            "sales_team_status" : "busy"
            } ,
            {
            "from_date" : "25-nov-2023",
            "to_date"   : "27-nov-2023",
            "description" : "corporate liquidity management ",
             "sales_team_status" : "busy"
            }
        ]
    }
    campaign = campaigns[city_name]
    campaign_withinfo = {}
    campaign_withinfo["city_name"] = city_name
    campaign_withinfo ["bank_name"] = bank_name
    campaign_withinfo ["detailed_reason"] = detailed_reason
    campaign_withinfo ["campaign"] = campaign
    JSON(campaign, expanded=True)  
    return json.dumps(campaign_withinfo)


In [5]:
def get_sales_intelligence(bank_name1 : str,bank_name2 : str):
    """returns the customers sales intelligence from CRM db"""
    bank_array = []
    bank_array.append(bank_name1)
    bank_array.append(bank_name2)    
    info = {
        "SBI" : {
            "name" : "State Bank of India",
            "is_our_customer" : "No",
            "sales_volume" : ""
        },
        "HDFC" : {
            "name" : "HDFC Bank",
            "is_our_customer" : "Yes",
            "sales_volume" : "5 million"
        },
        "PNB" : {
            "name" :  "Punjab National Bank",
            "is_our_customer" : "Yes",
            "sales_volume" : "50 million"
        },
        "ICICI" : {
            "name" : "ICICI Bank",
            "is_our_customer" : "Yes",
            "sales_volume" : "10 million"
        },
        "AXIS" : {
            "name" : "Axis Bank",
            "is_our_customer" : "Yes",
            "sales_volume" : "20 million"  
        }
    }
    crm_info = []
    for bank in bank_array:
        obj = info[bank]
        crm_info.append(obj)
        
    
    JSON(crm_info, expanded=True)  
    return json.dumps(crm_info)


In [6]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [ {"role" : "system", "content" : "You are market research guru . You have sharp analytical insights and you focus on meaningful insights and weeed out noise"},
        {"role": "user", "content": "what are top 2 indian banks from IT adoption perspective . Please do not worry that you do not have access to real time information.Format your answer using abbreviation only  "}]
    print_messages(messages)
    functions = [
        {
            "name": "get_sales_intelligence",
            "description": "Get the customer specific sales intelligence data from internal system",
            "parameters": {
                "type": "object",
                "properties": {
                    "bank_name1": {
                        "type": "string",
                        "description": "the names of the bank , example ICICI",
                    },
                    
                    "bank_name2": {
                        "type": "string",
                        "description": "the name of the second bank , example SBI",
                    }
                    
                },
                "required" : ["bank_name1","bank_name2"]

            },
        },
        {
            "name": "get_sales_campaign_running",
            "description": "Get the current sales campaign running will allow you to schedule future",
            "parameters": {
                "type": "object",
                "properties": {
                    "city_name": {
                        "type": "string",
                        "description": "the names of the city e.g Pune",
                    },
                       "bank_name": {
                        "type": "string",
                        "description": "the bank which is chosen",
                    },
                       "detailed_reason": {
                        "type": "string",
                        "description": "detailed reason for choosing bank over others",
                    },
                },
                "required" : ["city_name","bank_name","detailed_reason"]

            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions = functions,
        function_call = { "name": "get_sales_intelligence"}
    )
    print(response)
    response_message = response["choices"][0]["message"]
  

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_sales_intelligence": get_sales_intelligence,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        JSON(function_args,expanded=True)
        function_response = function_to_call(
            bank_name1=function_args.get("bank_name1"),
            bank_name2=function_args.get("bank_name2")
        )
        print (f"final formatted response from sales intelligence{function_response}")
        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        messages.append(
            {
                "role": "user",
                "content": "based on the all response , which bank out of these 2 banks should be target of our marketing campaign and why . Suggest a city where we can hold marketing campaign ",
            }
        )   
        print_messages(messages)
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            functions = functions,
            function_call = { "name": "get_sales_campaign_running"}
        )  # get a new response from GPT where it can see the function response
        
        response_message = second_response["choices"][0]["message"]
  

    # Step 4: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 5: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_sales_campaign_running": get_sales_campaign_running,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        JSON(function_args,expanded=True)
        function_response = function_to_call(
           city_name=function_args.get("city_name") ,
           bank_name=function_args.get("bank_name"),
           detailed_reason=function_args.get("detailed_reason")
           )
        print (f"final formatted response from sales campaign")
        json_object = json.loads(function_response)
        json_formatted_str = json.dumps(json_object, indent=2)
        event = "sales campaign"
        color_prefix =  '\033[31m' #RED
        print(f"{color_prefix}\n[{event}]\n{json_formatted_str}")
        
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    messages.append(
           {
            "role": "user",
             "content": "Display best suitable schedule for 2 days excluding current campaigns where sales team is engaged . please exclude diwali holidays and weekend ",
           }
        )
  
    third_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages
        )
                   
        
    return third_response


In [7]:
third_response = run_conversation()


[system]
You are market research guru . You have sharp analytical insights and you focus on meaningful insights and weeed out noise

[user]
what are top 2 indian banks from IT adoption perspective . Please do not worry that you do not have access to real time information.Format your answer using abbreviation only  
{
  "id": "chatcmpl-8AMcVSO9ztGr5c5W6qsodMPewy6ez",
  "object": "chat.completion",
  "created": 1697480923,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_sales_intelligence",
          "arguments": "{\n  \"bank_name1\": \"HDFC\",\n  \"bank_name2\": \"ICICI\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 216,
    "completion_tokens": 23,
    "total_tokens": 239
  }
}
final formatted response from sales intelligence[{"name": "HDFC Bank", "is_our_customer": "Yes", "sales_volume":

In [8]:
print(third_response)

{
  "id": "chatcmpl-8AMcnvj7NJ5CkaZfh7h3QuT8xe2Ax",
  "object": "chat.completion",
  "created": 1697480941,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Based on the information provided, the best suitable schedule for a 2-day marketing campaign in Mumbai targeting ICICI Bank, excluding current engagements and considering the exclusion of Diwali holidays and weekends, is as follows:\n\n1. From Date: 18-Nov-2023 (Thursday)\n   To Date: 19-Nov-2023 (Friday)\n   Description: \"Digital transformation in banking industry\"\n   Sales Team Availability: Available\n\nPlease note that this schedule is subject to change based on the availability of the sales team and other factors."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 492,
    "completion_tokens": 108,
    "total_tokens": 600
  }
}
